# Project 2: Graph Analysis
* Released: 04/16
* Due: 05/02 10AM
* Value: 5% of your grade
* Max team of 2

Many graph analysis compute network centrality, density, shortest paths, and other path-based statistics about a graph.  It may seem that writing a one-off Python script is a good way to perform this analysis, but it turns out that SQL is pretty great at doing this type of analysis!  

## Background of the Data
For this assignment, you will do a graph analysis to analyze Tweets from [**Twitter Elections Integrity Dataset**](https://about.twitter.com/en_us/values/elections-integrity.html#data).

- Justice Department charged 13 Russian nationals with interfering in American electoral and political processes. The defendants worked for a well-funded “troll factory” called the Internet Research Agency(**IRA**).
- IRA ran a campaign to sow disinformation and discord into American politics via social media (mostly twitter).
- Dataset includes information from 3613 accounts believed to be connected to the Russian Internet Research Agency.

#### The Twitter IRA dataset

In reality, the twitter dataset isn't as neat as the above example.  Instead it contains the following attributes.

```
tweetid                   # tweet id
userid                    # user id (hashed for users which had fewer than 5,000 followers) 
user_display_name         # name of user (same as userid for anonymized users)
user_screen_name          # the Twitter handle of the user
user_reported_location    # self-reported location
……
```

Please check [Twitter Elections Integrity Datasets Readme](https://storage.googleapis.com/twitter-election-integrity/hashed/Twitter_Elections_Integrity_Datasets_hashed_README.txt) for full description


### Important BigQuery Reference

[**BigQuery Standard SQL Functions & Operators API Reference**](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators)

## Refresher

You will write queries or short Python programs to answer the following questions about the dataset.  

In the simple case, graphs have the following schema:

        nodes(id int primary key, <attributes>)
        edges(
          src int NOT NULL references nodes(id),
          dst int NOT NULL references nodes(id),
          <attributes
        )

Recall that in graph analysis, you are interested in finding neighbors of nodes or paths between nodes.    Following an edge in the graph corresponds to a JOIN.  For example, the following finds all neighbors of node id 2:

        SELECT dst FROM edges WHERE src = 2;

Similarly, if we have a table `goodnodes` that contains IDs of nodes that we are interested in, the following query finds neighbors of these good nodes:

        SELECT dst FROM edges, goodnodes WHERE edges.src = goodnodes.id;

## **Before you begin**


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.
4. Follow [this instruction](https://github.com/w4111/project2_s19/blob/master/Project%20Setup%20Instructions.pdf) to create dataset which is used to store your temperary table


### Provide your and your teammate UNI

In [0]:
# Your columbia uni that is used in SSOL
My_UNI = "cu1111"
My_Teammate_UNI = "cu1111"

print("My UNI is {u1}, My teammate UNI is {u2}".format(u1=My_UNI, u2 = My_Teammate_UNI))

My UNI is cu1111, My teammate UNI is cu1111


### **Provide your credentials to the runtime**

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

### Test your settings and check the IRA dataset

Please fill in Google Project ID below

In [0]:
%%bigquery --project your_project_id
SELECT *
FROM `project2-236400.twitter.IRA` 
LIMIT 5

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,721935043237584897,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,3,27,[],[],,
1,759003437597990912,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,3,3,24,15,[],[],,
2,761963212304740352,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,1,5,11,[],[],,
3,704951387982065665,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,2,6,16,[],[],,
4,708289771030888449,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,[],[],[106111547],


## Question 1

Find the `id` of Tweets that contain both the phrase "MakeAmericaGreatAgain" **and** "Trump" (both **case-insensitive**) .  

For example:

* "#VoteTrump and lets all help #Trump #makeamericagreatagain" is a match
* "This was our moment. Together, we will make america great again!" is not a match

Your answer should be a single query containing the columns:
- id (id of the tweets)
- text (text of the tweets)

Just like ``testquery``, you should return the output of ``job.results()`` in a list.

###Write your SQL below

In [0]:
%%bigquery --project your_project_id

## Question 2
"**MakeAmericaGreatAgain**" (often abbreviated as MAGA) is a campaign slogan used in American politics that was popularized by Donald Trump in his successful 2016 presidential campaign.

Let's Find out Top 5 month gets the maximum mention of this slogan
"MakeAmericaGreatAgain"(**case-insensitive**) in `tweet_text` .

Your answer should be a single query containing the columns:
- year
- month
- num (total number of slogan for the month)

Just like ``testquery``, you should return the output of ``job.results()`` in a list.

###Write your SQL below

In [0]:
%%bigquery --project your_project_id


## Question 3   
**Let's construct the Graph!**

The original and retweet tweets can be looked at as a directed edge and hence we can look at it as a directed graph where each row is an edge between a tweeter and the user who retweet them. Create a table “Graph” with column names src and dst which stores the edge list of the graph. You must store only the distinct edges in the table.

Your table should contain the following columns:
- src (userid)
- dst (retweet_userid)


Also, one user might retweet another user more than one time. In this case, you should only save the edge once (i.e. only one row in the graph table).

You must save this table since you will be using it for the next few questions.

To save a table, there are two options:
1. Use the API to save job results in a table, as in the example function ``savetable()`` in the starter code. The dataset_id refers to the name of the dataset which you created and the table_ref variable specifies the name of the table to which the results will be saved into. For further reference, you can check the following link: [Saving Results to Table](https://cloud.google.com/bigquery/docs/writing-results).
2. Issue a regular query using ``CREATE OR REPLACE TABLE table_name_here AS SELECT ...``


###Write your SQL below

In [0]:
%%bigquery --project your_project_id


## Question 4
The indegree of a node in a directed graph is defined as the number of edges which are incoming on the node. Similarly, the outdegree of a node in a directed graph is defined as the number of edges which are outgoing from the node. For more information, you can read - [Indegree and Outdegree](https://en.wikipedia.org/wiki/Directed_graph#Indegree_and_outdegree)

Using this information, find out from the GRAPH table which user has the highest indegree and which user has the highest outdegree.

Your answer should be a single query containing the columns:
- max_indegree
- max_outdegree

###Write your SQL below

In [0]:
%%bigquery --project your_project_id


## Question 5
Let us define 4 categories of Twitter users. We will use the average number of likes a user gets on his/her tweets as the first metric and the number of times they are mentioned by other users in tweets (i.e. indegree) as the second metric. Then we can classify each user as follows:
- High indegree, high average number of likes 
- High indegree, low average number of likes
- Low indegree, high average number of likes
- Low indegree, low average number of likes

We will refer to the 'low indegree, low average likes' category of users as "unpopular" users and 'high indegree, high average likes' category of users as "popular" users. 

We define the indegree and average number of likes to be high or low based on the rules below:
1) If indegree < avg(indegree) of all the nodes in the graph then indegree is said to be low for the user, else it is considered high.
2) If the average number of likes for user < average number of likes for all the nodes in the graph, then the average number of likes is said to be low for the user; else it is considered high.

Now, you need to find the conditional probability, that given an unpopular user, what is the probability that they retweet tweets from a popular user i.e. find P(retweet popular user | unpopular user).

Your answer should be a query containing the column:
- popular_unpopular (conditional probability P(retweet popular user | unpopular user)

###Write your SQL below

In [0]:
%%bigquery --project your_project_id


## Question 6
Given a graph G = (V, E), a “triangle” is a set of three vertices that are mutually adjacent in G i.e. given 3 nodes of a graph A, B, C there exist edges A->B, B->C and C->A which form a triangle in the graph. From the graph table which you created above, find out the number of triangles in the graph.

For the first part, your answer should be a single query containing the column:
- no_of_triangles

Just like ``testquery``, you should return the ouput of ``job.results()`` in a list.

###Write your SQL below

In [0]:
%%bigquery --project your_project_id


## Question 7
The PageRank algorithm is used to rank the importance of nodes in a graph. It works by counting the number of edges incident to a node to determine how important the node is. The underlying assumption is that more important nodes are likely to receive more links from other nodes. Find the top 100 nodes with the highest PageRank in the graph.
Hint: It is not possible to use "WITH RECURSIVE" on BigQuery. You must develop a iterative implementation for PageRank (like the BFS example mentioned below).

You must run the algorithm for 20 iterations and your output table should contain the following columns:
- twitter_username (the twitter_username of the user)
- page_rank_score

Just like ``testquery``, you should return the output of ``job.results()`` in a list.

You must implement only the simplified version of the PageRank algorithm. 
This algorithm works as follows - Assume a small universe of four web pages: A, B, C and D. PageRank is initialized to the same value for all pages since we assume a probability distribution between 0 and 1 as the PageRank for each node. Hence the initial value for each page in this example is 0.25. If the only links in the system were from pages B->A, C->A and D->A, each link would transfer 0.25 PageRank to A upon the next iteration, for a total of 0.75 i.e. PR(A) = PR(B) + PR(C) + PR(D). 

Now, suppose instead that we have the links B->C, B->A, C->A, D->A, D->B, D->C. Thus, upon the first iteration, page B would transfer half of its existing value, or 0.125, to page A and the other half, or 0.125, to page C. Page C would transfer all of its existing value, 0.25, to the only page it links to, A. Since D had three outbound links, it would transfer one third of its existing value, or approximately 0.083, to A. At the completion of this iteration, page A will have a PageRank of approximately 0.458.
PR(A)=PR(B)/2 + PR(C)/1 + PR(D)/3.

Thus, we can write the PageRank of A as:
PR(A)= PR(B)/L(B) + PR(C)/L(C) + PR(D)/L(D) where L(x) gives us the number of outbound links for any node x. 

In general, the PageRank value for a page u is dependent on the PageRank values for each page v contained in the set containing all pages linking to page u, divided by the number of links from page v. 
It is given by the formula: ![](https://www.geeksforgeeks.org/wp-content/ql-cache/quicklatex.com-aafd3a0d9f8bb8325cf2b41a4a839bbf_l3.svg)

To read more about PageRank, you can refer to the following link: [PageRank](http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf)


For this question, you will need to develop an iterative solution, i.e. your python code will act as a driver and issue multiple queries to BigQuery iteratively. As an example, we provided an iterative implementation of Breadth First Search on the starter code.

To execute 5 iterations using A as a start node, you can simply call ``bfs(client, 'A', 5)``.

The example saves the nodes visited at each iteration in a table ``distances``, along with their distance to the initial node. The function itself does not return any value (however, remind that you will be required to return values for Q7).

### Do not edit this function. This is for helping you develop your own iterative PageRank algorithm.

In [0]:
project_id = '[your project ID]'

In [0]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

def bfs(client, start, n_iter):

    # You should replace dataset.bfs_graph with your dataset name and table name.
    q1 = """
        CREATE TABLE IF NOT EXISTS dataset.bfs_graph (src string, dst string);
        """
    q2 = """
        INSERT INTO dataset.bfs_graph(src, dst) VALUES
        ('A', 'B'),
        ('A', 'E'),
        ('B', 'C'),
        ('C', 'D'),
        ('E', 'F'),
        ('F', 'D'),
        ('A', 'F'),
        ('B', 'E'),
        ('B', 'F'),
        ('A', 'G'),
        ('B', 'G'),
        ('F', 'G'),
        ('H', 'A'),
        ('G', 'H'),
        ('H', 'C'),
        ('H', 'D'),
        ('E', 'H'),
        ('F', 'H');
        """

    job = client.query(q1)
    results = job.result()
    job = client.query(q2)
    results = job.result()

    # You should replace dataset.distances with your dataset name and table name. 
    q3 = """
        CREATE OR REPLACE TABLE dataset.distances AS
        SELECT '{start}' as node, 0 as distance
        """.format(start=start)
    job = client.query(q3)
    # Result will be empty, but calling makes the code wait for the query to complete
    job.result()

    for i in range(n_iter):
        print("Step %d..." % (i+1))
        q1 = """
        INSERT INTO dataset.distances(node, distance)
        SELECT distinct dst, {next_distance}
        FROM dataset.bfs_graph
            WHERE src IN (
                SELECT node
                FROM dataset.distances
                WHERE distance = {curr_distance}
                )
            AND dst NOT IN (
                SELECT node
                FROM dataset.distances
                )
            """.format(
                curr_distance=i,
                next_distance=i+1
            )
        job = client.query(q1)
        results = job.result()

In [0]:
bfs(client, 'A', 5)

Step 1...
Step 2...
Step 3...
Step 4...
Step 5...


###Write your SQL below

## Submission Instructions
* Click **File** on the colab menu, chose **Download .ipynb**

* Submit it via:
https://www.instabase.com/apps/file-submission/cmd/submit/05dd2e62-11b6-4219-9e59-2ede5ebc642b
